---

# **Notebook 9 : Comptage et Tri des Pays en Langue Latine dans les Données de Produit**

---

### **1. Importation des bibliothèques**

Cette première section est dédiée à l'importation des bibliothèques nécessaires pour la manipulation des fichiers et des données.


In [1]:
# Importation des bibliothèques
import os
import pandas as pd
import re

---

### **2. Définition de la fonction de comptage des pays**

Cette fonction lit les fichiers dans le dossier des chunks, filtre les noms de pays pour exclure ceux en caractères non-latins, compte les occurrences de chaque pays, et organise le résultat par ordre croissant.


In [ ]:
def compter_pays(input_directory):
    pays_counts = {}

    # Parcourir les fichiers dans le dossier des chunks
    for file_name in os.listdir(input_directory):
        input_file_path = os.path.join(input_directory, file_name)
        
        # Lire chaque chunk
        chunk_iter = pd.read_csv(input_file_path, sep=',', chunksize=10000, low_memory=False, on_bad_lines="skip")
        
        for chunk in chunk_iter:
            if 'countries' in chunk.columns:
                # Nettoyer les noms de pays en ne gardant que ceux avec des caractères latins
                chunk['countries_cleaned'] = chunk['countries'].apply(
                    lambda x: [country.strip() for country in re.split(r',|\s', str(x)) if re.match(r'^[a-zA-Z\s]+$', country)]
                )
                
                # Compter les occurrences des pays
                for countries_list in chunk['countries_cleaned']:
                    for country in countries_list:
                        if country:  # Ignorer les valeurs vides
                            pays_counts[country] = pays_counts.get(country, 0) + 1

    # Convertir le dictionnaire en DataFrame et trier par ordre croissant
    pays_df = pd.DataFrame(list(pays_counts.items()), columns=['Pays', 'Nombre de produits']).sort_values(by='Nombre de produits', ascending=True)
    
    return pays_df

### **3. Enregistrement du DataFrame trié dans un fichier CSV**

Cette section enregistre le tableau résultant, trié par ordre croissant, dans un fichier CSV pour une utilisation ultérieure.


In [ ]:
# Définir le répertoire des chunks et exécuter la fonction
input_directory = 'note-books/phase_1_data/07_chunk_no_redundant_columns'
pays_df = compter_pays(input_directory)

# Enregistrer le DataFrame dans un fichier CSV
output_csv_path = 'note-books/phase_1_data/08_data_countries/pays_counts_ordre_croissant.csv'
os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
pays_df.to_csv(output_csv_path, index=False)

print(f"Tableau des pays trié par ordre croissant enregistré dans {output_csv_path}")


### **Explications :**

- **Comptage et filtrage des pays** : La fonction `compter_pays` lit les données dans chaque chunk, filtre les noms de pays pour ne garder que ceux écrits avec des caractères latins, puis compte les occurrences de chaque pays.
- **Organisation des données** : Le tableau final est trié par ordre croissant de nombre de produits par pays pour une meilleure visualisation.
- **Enregistrement en CSV** : Le résultat est sauvegardé dans un fichier CSV, ce qui permet de l'exporter ou de l'utiliser dans des analyses ultérieures.

--- 
